# Part 2 舆情数据分析


# 1.scraping text data from twitter 

In [ ]:
#install modules
!pip install selenium
!pip time
!pip

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np

# need to download the chromedriver
browser=webdriver.Chrome()
#construt the searching url
base_url=u"https://twitter.com/search?q="
query=u"gold%20futures"
url=base_url+query

#parsing text from the twitter pages, webpage will automatically pull down
browser.get(url)
time.sleep(1)
body=browser.find_element_by_tag_name("body")
for _ in range(10000):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.01)

print("done!")
#since my computer is quite slow and have limited storage, I just collect the past 3 months tweets about gold futures


done!


**fetch the tweets text and time**

In [250]:
#get the tweets as well as the post time
tweets=browser.find_elements_by_class_name("tweet-text")
time_post=browser.find_elements_by_class_name("time")


In [251]:
#see how many posts we have crawled
len(tweets)
len(time_post)

3785

In [287]:
#put the posting text, posting dates into 2 lists
tw=[]
for tweet in tweets:
    tw.append(tweet.text)
tp=[]
for time in time_post:
    tp.append(time.text) 

In [252]:
# put the twitter text and dates into the same dataframe called:ndf
ndf=pd.DataFrame([tp,tw]).T
ndf=ndf.ix[0:3649,:]# remove the last few rows which might be wrong 


# 2.分词处理及清理

In [288]:
# split sentences in the text to single words,
#get lowercase of the words, remove those less than three characters 
#combine those words series by columns
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
tws_clean= pd.DataFrame(tokenizer.tokenize(tw[0]))
for  i in range(3649):
    tws=tw[i+1] 
    tws_new = tokenizer.tokenize(tws)
    tws_new1= [word.lower() for word in tws_new if len(word)>3]
    tws_new2 = pd.DataFrame(tws_new1)
    tws_clean=pd.concat([tws_clean,tws_new2],axis=1)
    


In [289]:
# tem is the clean words series
tem=tws_clean.T
tem.shape
tws_clean.head()
tem.index=ndf.index
tem

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,Macron,s,Victory,For,The,World,Sparks,Panic,Dump,Of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gold,futures,soon,test,support,this,will,interesting,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,gold,futures,shorting,attacks,https,goldbroker,news,gold,futures,shorting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gold,major,support,1251,with,resistance,1263,gold,futures,stocks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pathchat,weekend,futures,update,2017,bob_iaccino,mgarnold,sp500,nasdaq,gold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,panini,gold,standard,kawhi,leonard,golden,futures,auto,autograph,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,futures,markets,mildly,lower,gold,post,comey,ouster,http,2q1qezk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ahead,when,trading,stocks,forex,futures,gold,more,software,page,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,stock,futures,decline,gold,after,director,comey,fired,seeking,alpha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,open,interest,gold,futures,fallen,since,437k,heading,right,direction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3.sentiment analysis：
**Prepare the sentiment dictionary**

In [261]:
# prepare the sentiment dictionary,here we have negative words and positive
#words, and convert them to a list
negatives=pd.read_csv("negatives.csv")
negative = [neg.strip().lower() for neg in negatives["Negative"]]
#load the negative and positive dictionary
positives=pd.read_csv("positives.csv")
positives
positive = [pos.strip().lower() for pos in positives["Positive"]]




**计算每条tweet的sentiment指标的函数： 即词性计数的函数**

In [260]:
  
#countPos(cleantext, positive): counts the number of words contained both in the post and in the positive dictionary.
def countPos(cleantext, positive):
    """
    counts positive words in cleantext
    """
    pos = [word for word in cleantext if word in positive]
    return len(pos)   

#countNeg(cleantext, negative): counts the number of words contained both in the post and in the negative dictionary.
def countNeg(cleantext, negative):
    """
    counts negative words in cleantext
    """
    negs = [word for word in cleantext if word in negative]
    return len(negs)

#**function to do sentiment anlysis：
#getSentiment(cleantext, negative, positive): returns the difference between positive and negative words in a post.
def getSentiment(cleantext, negative, positive):
    """
    counts negative and positive words in cleantext and returns a score accordingly
    """
    
    return (countPos(cleantext, positive) - countNeg(cleantext, negative))

** calculate the sentiment score for each twitter post

In [265]:
score_for_all=[]
for i  in range(3650):
        test=tem.ix[i+1,:]
        score=getSentiment(test, negative, positive)
        score_for_all.append(score)
#put the sentiment score to the original dataframe ndf
ndf["sentiment"]=score_for_all
#

 0    2265
 1     568
-1     428
-2     170
 2     144
-3      29
 3      21
-4      13
 4      10
 5       1
-5       1
Name: sentiment, dtype: int64

#since we have limited words in the dictionary, we must miss a lot info
#in the posts, thus we should value more on those tweets which have negative 
#or positive impact, we then sum the score for each day and get the final 
#sentiment score for each trading date

In [ ]:
# pivot table
ndf1=ndf
ndf1.columns=["time","tweets","sentiment"]
ndf1.ix[5:12,"time"]=ndf1.ix[12,"time"]#fix some bugs

ndf2=pd.pivot_table(ndf1,index=["time"],values=["sentiment"],aggfunc=np.sum)
ndf2

In [274]:
# convert date lables to datetime
from datetime import datetime
t=ndf2.index
t
Time=[]
for i in range(93):
    date=timestring.Date(t[i],'%y/%b/%d')
    Time.append(date)


In [276]:
#rename the index of ndf2  as dates and sort by index
ndf2.index=Time
ndf2.sort_index()
#replace those sentiment score with -1 or 1
ndf2.ix[ndf2["sentiment"]<0,"sentiment"]=-1
ndf2.ix[ndf2["sentiment"]>0,"sentiment"]=1 


In [283]:
ndf2=ndf2.sort_index()
ndf2.to_csv("sentiment.csv")# here ndf2 is the most sentiment variable 